#*Project 1: Morse Code*#

In [ ]:
!pip install gradio pandas simpleaudio matplotlib

In [ ]:
import gradio as gr
import pandas as pd
import simpleaudio as sa
import matplotlib.pyplot as plt
import os

# Morse Code Mapping
morse_code_map = {
    "0": "-----", "1": ".----", "2": "..---", "3": "...--", "4": "....-",
    "5": ".....", "6": "-....", "7": "--...", "8": "---..", "9": "----.",
    "A": ".-", "B": "-...", "C": "-.-.", "D": "-..", "E": ".", "F": "..-.",
    "G": "--.", "H": "....", "I": "..", "J": ".---", "K": "-.-", "L": ".-..",
    "M": "--", "N": "-.", "O": "---", "P": ".--.", "Q": "--.-", "R": ".-.",
    "S": "...", "T": "-", "U": "..-", "V": "...-", "W": ".--", "X": "-..-",
    "Y": "-.--", "Z": "--..", " ": "/"
}

# Reverse Mapping
morse_to_text_map = {v: k for k, v in morse_code_map.items()}

# Encode Text to Morse Code
def generate_morse(input_text):
    input_text = input_text.strip().upper()
    try:
        morse_code = " ".join(morse_code_map[char] for char in input_text)
        return morse_code
    except KeyError:
        return "Error: Invalid characters in input. Only letters, numbers, and spaces are supported."

# Decode Morse Code to Text
def interpret_morse(input_morse):
    input_morse = input_morse.strip()
    try:
        words = input_morse.split(" / ")
        decoded_words = ["".join(morse_to_text_map[char] for char in word.split()) for word in words]
        return " ".join(decoded_words)
    except KeyError:
        return "Error: Invalid Morse Code entered. Please check input format."

# Generate Morse Code Sound
def play_morse_audio(morse_code):
    dot = sa.WaveObject.from_wave_file("dot.wav")
    dash = sa.WaveObject.from_wave_file("dash.wav")
    for symbol in morse_code:
        if symbol == ".":
            dot.play().wait_done()
        elif symbol == "-":
            dash.play().wait_done()
        elif symbol == " ":
            continue

# Logging Sentiment Data
log_file = "morse_log.csv"
def log_conversion(input_text, output_text, mode):
    df = pd.DataFrame([[mode, input_text, output_text]], columns=["Mode", "Input", "Output"])
    df.to_csv(log_file, mode='a', header=not os.path.exists(log_file), index=False)

# Data Visualization - Frequency Analysis
def visualize_logs():
    if os.path.exists(log_file):
        df = pd.read_csv(log_file)
        counts = df["Mode"].value_counts()
        counts.plot(kind='bar', title='Usage Frequency')
        plt.show()
    else:
        return "No data to display."

# Gradio Interface
def morse_interface(text, mode, play_audio):
    if mode == "Text to Morse":
        result = generate_morse(text)
        if play_audio:
            play_morse_audio(result)
    else:
        result = interpret_morse(text)
    log_conversion(text, result, mode)
    return result

iface = gr.Interface(
    fn=morse_interface,
    inputs=[
        gr.Textbox(label="Enter Text / Morse Code"),
        gr.Radio(["Text to Morse", "Morse to Text"], label="Select Mode"),
        gr.Checkbox(label="Play Morse Code Audio")
    ],
    outputs="text",
    title="Morse Code Encoder & Decoder",
    description="Convert text to Morse code and vice versa. Supports spaces, numbers, and sound output.",
)

iface.launch()
